# Linear Regression Code Along

This notebook is the reference for the video lecture on the Linear Regression Code Along. Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [4]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank Tunnel\nWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.655651149166752, Time on Website=39.57766801952616, Length of Membership=4.082620632952961, Yearly Amount Spent=587.9510539684005)

In [6]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning 

In [7]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [10]:
output = assembler.transform(data)

In [11]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [12]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [13]:
final_data = output.select("features",'Yearly Amount Spent')

In [14]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [15]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                345|
|   mean| 497.54893542705685|
| stddev|  78.24049760468148|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [16]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                155|
|   mean|  503.2428155287786|
| stddev|  81.77563315978306|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+



In [17]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [18]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

In [19]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.70263697579585,38.65046018406829,0.6859980626804102,61.504914727599605] Intercept: -1058.870371387049


In [20]:
test_results = lrModel.evaluate(test_data)

In [21]:
# Interesting results....
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-11.653936846559986|
| 10.833351277242514|
| 7.0423994634707014|
| -5.884313855579251|
| 22.519160877904596|
| 0.2023115656841128|
| -8.015701371835007|
| -5.081233107350329|
| -9.462208519369312|
|-1.8969582885989666|
| 18.134160397820324|
| -7.353663369118635|
| -5.908364387048948|
| -10.94133858365899|
|-3.1349960140105395|
|-1.9183950370692742|
| -2.504705810423957|
| -9.226932575303806|
|  11.80103875878774|
| 0.0438372537286682|
+-------------------+
only showing top 20 rows



In [26]:
test_results.r2

0.9874262331447837

In [27]:
test_results.rootMeanSquaredError

9.14010381158938

In [22]:
unlabeled_data = test_data.select('features')

In [23]:
predictions = lrModel.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 331.5828066497536|
|[30.7377203726281...|450.94739091898737|
|[30.9716756438877...|  487.596210293422|
|[31.0613251567161...|493.43977191348085|
|[31.2834474760581...| 569.2619285477629|
|[31.3895854806643...| 409.8672994942988|
|[31.4474464941278...|  426.618443467059|
|[31.5171218025062...|280.99965375773604|
|[31.5261978982398...| 418.5567347117071|
|[31.5761319713222...| 543.1235422779273|
|[31.6005122003032...| 461.0386910932766|
|[31.7207699002873...| 546.1285968471416|
|[31.7242025238451...|509.29625167500944|
|[31.8093003166791...| 547.7132379465002|
|[31.8124825597242...|395.94534099780776|
|[31.8186165667690...| 448.3370684072049|
|[31.8530748017465...|461.78982927277593|
|[31.8854062999117...| 399.3302055477793|
|[31.9096268275227...| 551.6449969144514|
|[32.0047530203648...|463.70214386690077|
+--------------------+------------

In [25]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.14010381158938
MSE: 83.5414976866307


Excellent results! Let's see how you handle some more realistically modeled data in the Consulting Project!